In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [ ]:
pip install fbprophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Created wheel for fbprophet: filename=fbprophet-0.7.1-py3-none-any.whl size=6639047 sha256=efec6d5f8853fb30d548cd75ed38756997d5fc4b6d75eb9c5942bb8b012e7434
  Stored in directory: /root/.cache/pip/wheels/cd/a1/12/db63ff624de492fe6cccf676091a0860fdde2ffde4bc3280e2
Successfully built fbprophet


In [ ]:
pip install pystan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install --no-cache-dir -I pystan==2.19.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67.3 MB 1.4 MB/s 
     |████████████████████████████████| 1.9 MB 40.0 MB/s 
     |████████████████████████████████| 15.7 MB 40.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1 requires cmdstanpy>=1.0.1, but you have cmdstanpy 0.9.5 which is incompatible.


In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


# **Getting The Data**

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2019-01-01'
eth_df = yf.download('ETH-USD',start_date, today)
eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-15,"$1,635.08","$1,648.95","$1,466.14","$1,471.69","$1,471.69",26946275878
2022-09-16,"$1,471.93","$1,480.56","$1,415.58","$1,432.45","$1,432.45",16764804299
2022-09-17,"$1,432.88","$1,473.06","$1,415.04","$1,469.74","$1,469.74",10798098671
2022-09-18,"$1,469.71","$1,469.71","$1,331.49","$1,335.33","$1,335.33",15762284723
2022-09-19,"$1,335.27","$1,388.27","$1,287.42","$1,377.54","$1,377.54",18712714223


# **Explainatory Data Analysis**

In [ ]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1776 entries, 2017-11-09 to 2022-09-19
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1776 non-null   float64
 1   High       1776 non-null   float64
 2   Low        1776 non-null   float64
 3   Close      1776 non-null   float64
 4   Adj Close  1776 non-null   float64
 5   Volume     1776 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 97.1 KB


In [ ]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
eth_df.columns



Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
df = eth_df[["Date", "Open"]]
new_names = {
    "Date": "ds", 
    "Open": "y",
}
df.rename(columns=new_names, inplace=True)

In [ ]:
df.tail()

,ds,y
1771,2022-09-15,"$1,635.08"
1772,2022-09-16,"$1,471.93"
1773,2022-09-17,"$1,432.88"
1774,2022-09-18,"$1,469.71"
1775,2022-09-19,"$1,335.27"


In [ ]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

# **Prophet model**

In [ ]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2136,2023-09-15
2137,2023-09-16
2138,2023-09-17
2139,2023-09-18
2140,2023-09-19


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2136,2023-09-15,$-415.03,"$-1,774.22",$933.01
2137,2023-09-16,$-414.37,"$-1,691.05",$926.51
2138,2023-09-17,$-418.37,"$-1,711.45",$933.71
2139,2023-09-18,$-421.69,"$-1,781.66",$904.18
2140,2023-09-19,$-423.66,"$-1,752.16",$890.32


In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()

1435.458481726009

In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)